In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.metrics import roc_curve, auc, classification_report, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, plot_confusion_matrix

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
df = drive.CreateFile({'id': '179nL1l5rwe1505WtJwRwemCiw9BaV0S3'})
df.GetContentFile('conditions_df.csv')
df = pd.read_csv('conditions_df.csv')

In [ ]:
,# Removing date column for simplicity - do not want to work on time series data :)
# Dropping redundant classification columns and Stn Id to improve detecting important weather features
to_drop = ['Date', 'Stn Id', 'Stn Name', 'CIMIS Region']
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
print("Dataset has {} entries and {} features".format(*df.shape))

In [ ]:
# Important to note that "Notes" column is only for target columns
# possible leakage => need to drop
print(df[~df['Notes'].isna()]['Target'].value_counts())
df.drop('Notes', axis=1, inplace=True)

In [ ]:
df

In [ ]:
missing_values = 100 * df.isna().sum()/len(df)
missing_values = missing_values.reset_index()
missing_values.columns = ['feature', '%_nan_values']

fig = px.bar(missing_values, y='%_nan_values', x='feature',
             title='Missing Values', template='ggplot2')
fig.show();

In [ ]:
temp = df[df['Target']==1]
missing_values = 100 * temp.isna().sum()/len(temp)
missing_values = missing_values.reset_index()
missing_values.columns = ['feature', '%_nan_values']

fig = px.bar(missing_values, y='%_nan_values', x='feature',
             title='Missing Values', template='ggplot2')
fig.show();

In [ ]:
# A lot of nulls in our Target column - for simplicity, filling with averages
# Don't drop - losing a lot of info on small imbalanced target class!
df = df.fillna(df.mean())

In [ ]:
def plot_subgroup_hist(df, class1, class2):
    '''
    Displays information of 2 classes of a data set
    
    PARAMETERS
    ----------
        df: dataframe
        class1: dataframe of 1 class
        class2: dataframe of 2nd class
    '''
    dim = len(df.columns)
    fig, axs = plt.subplots(3, int(dim/3), figsize=(12, 6))
    for i, col_name in enumerate(class1.columns):
        bins = np.linspace(df[col_name].min(), df[col_name].max(), 20)
        height, binz = np.histogram(class1[col_name], bins=bins, density=True)
        bp1 = axs[i%3][i//3].bar(bins[:-1], height, .5*(bins[1]-bins[0]),
                     alpha=0.5, label="Fire", color='r')
        height, binz = np.histogram(class2[col_name], bins=bins, density=True)
        bp2 = axs[i%3][i//3].bar(bins[:-1]+.5*(bins[1]-bins[0]), height,
                     .5*(bins[1]-bins[0]), color='b', alpha=.5)
        axs[i%3][i//3].set_title(col_name)
        axs[i%3][i//3].legend((bp1[0], bp2[0]), ("Fire", "No Fire"), loc='best')

    plt.tight_layout()

    return fig, axs

In [ ]:
df_fire = df[df['Target'] == 1]
df_no_fire = df[df['Target'] == 0]

plot_subgroup_hist(df, df_fire, df_no_fire)
#plt.savefig('eda_histograms.png');

In [ ]:
(df.Target.value_counts()/len(df))*100
# quite imbalanced...

In [ ]:
# Next steps are attempting different class-balancing techniques
# OR adjusting class_weight in model hyperparameters

In [ ]:
y = df.pop('Target')

In [ ]:
scaled_df = StandardScaler().fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=df.columns)

## Some fun with oop - experimenting!

In [ ]:
class Fire(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def split(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, stratify=self.y)

    def predict(self, model):
        self.model = model
        kf = KFold(n_splits=5)
        
        accuracy = []
        precision = []
        recall = []
        f1 = []

        for train_index, test_index in kf.split(self.X_train):
          X_train_split, X_test_split = self.X_train.iloc[train_index], self.X_train.iloc[test_index]
          y_train_split, y_test_split = self.y_train.iloc[train_index], self.y_train.iloc[test_index]
          self.model.fit(X_train_split, y_train_split)
          self.pred = self.model.predict(X_test_split)

          assess = lambda method, val=y_test_split, pred=self.pred: method(val, pred)

          accuracy.append(assess(accuracy_score))
          precision.append(assess(precision_score))
          recall.append(assess(recall_score))
          f1.append(assess(f1_score))
        
        return np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(f1)
      
    def get_rates(self):
        self.proba = self.model.predict_proba(self.X_test)
        self.proba = self.proba[:,1]
        self.fpr, self.tpr, self.thresholds = roc_curve(self.y_test, self.proba)
        self.auc = auc(self.fpr, self.tpr)

        return self.fpr, self.tpr, self.auc

    def cm(self):
        return plot_confusion_matrix(self.model, self.X_test, self.y_test, cmap=plt.cm.Purples , normalize='true')
    
    def plot_roc(self, ax, model):
        if model == 'knn':
          ax.plot(self.fpr, self.tpr, color='orange', label=f'{model}: {round(self.auc, 4)}')
          return ax
        elif model == 'forest':
          ax.plot(self.fpr, self.tpr, color='green', label=f'{model}: {round(self.auc, 4)}')
          return ax
        elif model == 'boost1':
          ax.plot(self.fpr, self.tpr, color='red', label=f'{model}: {round(self.auc, 4)}')
          return ax
        else:
          ax.plot(self.fpr, self.tpr, color='purple', label=f'{model}: {round(self.auc, 4)}')
          return ax

    def plot_importance(self):
        return plot_importance(self.model, max_num_features=15)

In [ ]:
knn = KNeighborsClassifier()

data_knn = Fire(scaled_df, y)
data_knn.split()
data_knn.predict(knn)
data_knn.get_rates()

data_knn.cm()
plt.title('K Nearest Neighbors Confusion Matrix')
plt.grid(False)
#plt.savefig('knn_cm.jpeg')
plt.show();

In [ ]:
forest = RandomForestClassifier(class_weight='balanced')

data_forest = Fire(scaled_df, y)
data_forest.split()
rf_accuracy, rf_precision, rf_recall, rf_f1 = data_forest.predict(forest)
data_forest.get_rates()

data_forest.cm()
plt.title('Random Forest Confusion Matrix')
plt.grid(False)
#plt.savefig('forest_cm.jpeg')
plt.show();

In [ ]:
boost = XGBClassifier(class_weight='balanced')

data_boost = Fire(scaled_df, y)
data_boost.split()
gb_accuracy, gb_precision, gb_recall, gb_f1 = data_boost.predict(boost)
data_boost.get_rates()

data_boost.cm()
plt.title('XGBoost Confusion Matrix')
plt.grid(False)
#plt.savefig('boost_cm.jpeg')
plt.show()

# Feature Importances
data_boost.plot_importance()
#plt.savefig('feature_importances.jpeg')
plt.show()

In [ ]:
# ROC CURVE
fig, ax = plt.subplots(figsize=(10,10))
data_knn.plot_roc(ax, model='knn')
data_forest.plot_roc(ax, model='forest')
data_boost.plot_roc(ax, model='boost')
ax.set_title('ROC Curves')
ax.plot([0,1], [0,1], color ='k', linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(loc="lower right")
#plt.savefig('untrained_model_comparison.png')
fig.show();

## Fun oop over...

In [ ]:
def roc_curve(probabilities, labels):
    '''
    INPUT: numpy array, numpy array
    OUTPUT: array, array, array

    Take a numpy array of the predicted probabilities and a numpy array of the
    true labels.
    Return the True Positive Rates, False Positive Rates and Thresholds for the
    ROC curve.
    '''
    # tpr = tp / tp+fn
    # fpr = fp / fp+tn
    
    df = pd.DataFrame({'probabilities': probabilities, 'y': labels})
    df.sort_values('probabilities', inplace=True)

    actual_p = df.y.sum()
    actual_n = df.shape[0] - df.y.sum()
    
    df['tn'] = (df.y == 0).cumsum()
    df['fn'] = df.y.cumsum()
    df['fp'] = actual_n - df.tn
    df['tp'] = actual_p - df.fn
    
    df['fpr'] = df.fp/(df.fp + df.tn)
    df['tpr'] = df.tp/(df.tp + df.fn)
    df['precision'] = df.tp/(df.tp + df.fp)
    df['F1'] = 2*((df.tp/(df.tp + df.fp)) * (df.tp/(df.tp + df.fn)))/((df.tp/(df.tp + df.fp)) + (df.tp/(df.tp + df.fn)))
    df = df.reset_index(drop=True)
    return df

In [ ]:
def plot_roc(ax, df):
    ax.plot([1]+list(df.fpr), [1]+list(df.tpr), label="ROC")
    ax.plot([0,1],[0,1], 'k', label="random")
    ax.set_xlabel('FPR')
    ax.set_ylabel('TPR')
    ax.set_title('ROC Curve')
    ax.legend()

In [ ]:
def roc_continuous_curve(probabilities, alpha, labels):
    probabilities = np.where(probabilities > alpha, 1, 0)
    
    df = pd.DataFrame({'probabilities': probabilities, 'y': labels})
    df.sort_values('probabilities', inplace=True)

    actual_p = df.y.sum()
    actual_n = df.shape[0] - df.y.sum()
    
    df['tn'] = (df.y == 0).cumsum()
    df['fn'] = df.y.cumsum()
    df['fp'] = actual_n - df.tn
    df['tp'] = actual_p - df.fn
    
    df['fpr'] = df.fp/(df.fp + df.tn)
    df['tpr'] = df.tp/(df.tp + df.fn)
    df['precision'] = df.tp/(df.tp + df.fp)
    df['F1'] = 2*((df.tp/(df.tp + df.fp)) * (df.tp/(df.tp + df.fn)))/((df.tp/(df.tp + df.fp)) + (df.tp/(df.tp + df.fn)))
    df = df.reset_index(drop=True)
    return df

In [ ]:
def model_comparison(model_list, X_train, y_train, X_test, y_test):
    
    figure(figsize=(15, 10))

    for m in model_list:
        model = m # select the model
        model.fit(X_train, y_train) # train the model
        y_pred=model.predict(X_test) # predict the test data

    # Compute False postive rate, and True positive rate
        df = roc_curve(y_pred, y_test)

    # Calculate Area under the curve to display on the plot
        area_under_curve = roc_auc_score(y_test, y_pred)

    # Plot the computed values
        plt.plot(df.fpr, df.tpr, label=f"{model.__class__.__name__}, {round(area_under_curve, 2)}")

    # Custom settings for the plot 
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.title('ROC Curve')
    plt.xlabel('1-Specificity (False Positive Rate)')
    plt.ylabel('Sensitivity (True Positive Rate)')
    plt.legend();

In [ ]:
def gridsearch_with_output(estimator, parameter_grid, X_train, y_train):
    '''
    Gridsearches to hypertune estimator on training data.
    
    PARAMETERS
    ----------
        estimator: the type of model (e.g. RandomForestRegressor())
        paramter_grid: dictionary defining the gridsearch parameters
        X_train: 2d numpy array
        y_train: 1d numpy array
        
    RETURNS
    -------
        best parameters and model fit with those parameters
    '''
    model_gridsearch = GridSearchCV(estimator,
                                    parameter_grid,
                                    n_jobs=-1,
                                    verbose=10,
                                    cv=3,
                                    scoring='recall')
    model_gridsearch.fit(X_train, y_train)
    best_params = model_gridsearch.best_params_ 
    model_best = model_gridsearch.best_estimator_
    print("\nResult of gridsearch:")
    print("{0:<20s} | {1:<8s} | {2}".format("Parameter", "Optimal", "Gridsearch values"))
    print("-" * 55)
    for param, vals in parameter_grid.items():
        print("{0:<20s} | {1:<8s} | {2}".format(str(param), 
                                                str(best_params[param]),
                                                str(vals)))
    return best_params, model_best

In [ ]:
X_train, X_test,  y_train, y_test = train_test_split(scaled_df, y, stratify=y, test_size=.2)

In [ ]:
classification_models = [LogisticRegression(), DecisionTreeClassifier(class_weight='balanced'), RandomForestClassifier(class_weight='balanced'), GradientBoostingClassifier(class_weight='balanced')]

for model in classification_models:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(f'{model.__class__.__name__}\n')
  print(f'{classification_report(y_test, y_pred)}\n')

model_comparison(classification_models, X_train, y_train, X_test, y_test)

In [ ]:
parameter_grid = {
    'max_depth': [10, 30, 50, 70],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [2, 4],
    'min_samples_split': [2, 5],
    'n_estimators': [200, 400, 500]
}

rf_params, rf_gridsearch = gridsearch_with_output(RandomForestClassifier(class_weight='balanced'), parameter_grid, X_train, y_train)

In [ ]:
# Tuned RF
filename = 'tuned-rf.pkl'
pickle.dump(rf_gridsearch, open(filename, 'wb'))